# Insatlling all the necessary libraries

In [ ]:
pip install subprocess32

In [ ]:
pip install SpeechRecognition

In [ ]:
pip install Py-OS

In [ ]:
pip install wolframalpha

In [ ]:
pip install pyttsx3

In [ ]:
pip install wikipedia

In [ ]:
pip install DateTime

In [ ]:
pip install web-browser

In [ ]:
pip install requests

In [ ]:
pip install pywhatkit

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install ecapture

In [ ]:
pip install pyttsx3

In [ ]:
pip install PyAudio

In [ ]:
pip install transformers

In [ ]:
pip install newspaper3k

# Importing the libraries

In [ ]:
import os
import pyaudio
import subprocess32
import pyttsx3
import speech_recognition as sr
import pywhatkit
import datetime
import time
import wikipedia
import webbrowser
import requests
import ecapture as ec
from bs4 import BeautifulSoup
import ctypes
from transformers import T5ForConditionalGeneration, T5Tokenizer, BertTokenizer, pipeline

# Importing Natural Language Tool Kit  and Randon Library 
we import 'nltk' library for natural language procesing  
and importing 'random' library for selecting rondom movie name from the crated list

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import random

nltk.download('vader_lexicon')

we set a female voice for an assisant to communicate with user using *pyttsx3* library

In [ ]:
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

engine.setProperty('rate', 200)
# Set volume 0-1
engine.setProperty('volume', 1)

 adding a Audio in *speak* variable

In [ ]:
def speak(audio):
    engine.say(audio)
    engine.runAndWait()

# Summary generation and Question answering 

In [ ]:
# Initialize summarization model
tokenizer_sum = T5Tokenizer.from_pretrained('t5-base')
model_sum = T5ForConditionalGeneration.from_pretrained('t5-base')

# Initialize question answering model
tokenizer_qa = BertTokenizer.from_pretrained('bert-base-uncased')
question_answerer = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

def generate_summary(results):
    input_ids = tokenizer_sum.encode("summarize: " + results, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model_sum.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer_sum.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def answer_question(question, context):
    answer = question_answerer(question=question, context=context)
    return answer["answer"]

# Defining WishMe function

In [ ]:
def wishMe():
    hour = int(datetime.datetime.now().hour)
    if hour >= 0 and hour < 12:
        speak("Good Morning !")

    elif hour >= 12 and hour < 18:
        speak("Good Afternoon !")

    else:
        speak("Good Evening!")

    asname = ("zahra")
    speak("I am your Assistant")
    speak(asname)

# defining Username function to store username in the code tempararily

In [ ]:
def username():
    global uname
    speak("What should i call you Dear")
    uname = takeCommand()
    speak("Welcome Dear")
    speak(uname)
    speak("How can i Help you")

# importing pandas library to handle movies dataset

In [ ]:
import pandas as pd

In [ ]:
movies_df = pd.read_csv('movies.csv')


# performing sentiment analysis to check the user mood is posiive, negative or neutral

In [ ]:
# Define sentiment analysis function
def perform_sentiment_analysis(text):
    sia = SentimentIntensityAnalyzer()
    sentiment_score = sia.polarity_scores(text)['compound']

    if sentiment_score >= 0.05:
        return "Positive"
    elif sentiment_score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

# Splitting the dataset to recommand movie to user

In [ ]:
# Define entertainment suggestion function based on sentiment
def suggest_movie_based_on_sentiment(sentiment):
    if sentiment == "Positive":
        comedy_movies = movies_df[movies_df['genre'] == 'Comedy']
        animation_movies = movies_df[movies_df['genre'] == 'Animation']
        suitable_movies = pd.concat([comedy_movies, animation_movies])
    elif sentiment == "Negative":
        suitable_movies = movies_df[movies_df['genre'] == 'Horror']
    else:
        action_movies = movies_df[movies_df['genre'] == 'Action']
        biography_movies = movies_df[movies_df['genre'] == 'Biography']
        suitable_movies = pd.concat([action_movies, biography_movies])

    if not suitable_movies.empty:
        selected_movie = suitable_movies.sample(1)
        movie_name = selected_movie['name'].values[0]
        release_year = selected_movie['year'].values[0]
        return f"{movie_name} Released in {release_year}"
    else:
        return f"No suitable movies found for {sentiment.lower()} sentiment."


# Defining the functtion to take user commands as a query

In [ ]:
def takeCommand():
    r = sr.Recognizer()

    with sr.Microphone() as source:

        print("Listening...")
        r.pause_threshold = 0.5
        audio = r.listen(source)

    try:
        print("Recognizing...")
        query = r.recognize_google(audio, language='en-in')
        print(f"User said: {query}\n")

    except Exception as e:
        print(e)
        print("Unable to Recognize your voice.")
        return "None"
    return query

# defining the wikipedia section with summerrization and question answering model


In [ ]:
def wikipedia_sec():
    query = takeCommand()
    speak('Searching Wikipedia...')
    try:
        page = wikipedia.page(query)
        speak("According to Wikipedia")

        # Use the summarization and question-answering models
        summary = generate_summary(page.content)
        speak("Summarizing the content fetched from wikipedia")
        speak(summary)

        while True:
            speak("Do you have any questions about this topic? (Enter 'exit' to stop)")
            question = takeCommand()
            if question.lower() == 'exit':
                break
            if question:
                answer = answer_question(question, page.content)
                speak("Answer:")
                speak(answer)
            else:
                speak("No questions detected or invalid question.")

    except wikipedia.exceptions.DisambiguationError as e:
        print("The query is ambiguous, can you be more specific?")
    except wikipedia.exceptions.PageError:
        print("No Wikipedia page found for the query.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Defining the function to fetch the news from the website through API key

In [ ]:
def NewsFromBBC():
     
    query_params = {
      "source": "bbc-news",
      "sortBy": "top",
      "apiKey": "eb4d559681bb47279c97f3e936e5c421"
    }
    main_url = " https://newsapi.org/v1/articles"
 
    # fetching data in json format
    res = requests.get(main_url, params=query_params)
    open_bbc_page = res.json()
 
    # getting all articles in a string article
    article = open_bbc_page["articles"]
 
    # empty list which will 
    # contain all trending news
    results = []
     
    for ar in article:
        results.append(ar["title"])
         
    for i in range(len(results)):
         
        # printing all trending news
        X=print(i + 1, results[i])
 
    from win32com.client import Dispatch
    speak = Dispatch("SAPI.Spvoice")
    speak.Speak(results)  

# Defning the function where we created a while loop for continuous interaction with user

In [ ]:
def assistant():
    global Text1
    Text1 = ""
    while True:
        asname = ("zahra")
        
        query = takeCommand().lower()

        
        if 'wikipedia' in query:
            wikipedia_sec()
            
        elif 'suggest' in query:
            speak('Yes, of course, I can help you suggest movies according to your mood.')
            speak("How was your day")
            user_input_text = takeCommand()
            user_sentiment = perform_sentiment_analysis(user_input_text)
            suggested_movie = suggest_movie_based_on_sentiment(user_sentiment)
            speak("I think you are in")
            speak(user_sentiment)
            speak('mood')
            speak("i suggest you to see")
            speak(suggested_movie)
            Text1 += suggested_movie + "\n"
        
        elif 'open youtube' in query:
            speak("Here you go to Youtube\n")
            webbrowser.open("youtube.com")
            
        elif 'open google' in query:
            speak("Here you go to Google\n")
            webbrowser.open("google.com")
            

        elif 'open stackoverflow' in query:
            speak("Here you go to Stack Over flow.Happy coding")
            webbrowser.open("stackoverflow.com")

        elif 'play music' in query or "play song" in query:
            speak("Here you go with music")
            song = query.replace('play', '')
            speak('playing' + song)
            pywhatkit.playonyt(song)

        elif 'time' in query:
            time = datetime.datetime.now().strftime('%I:%M %p')
            speak("current time is" + time)
            

        elif 'how r u' in query:
            speak("I am fine, Thank you")
            speak("How are you, Dear")

        elif 'fine' in query:
            speak("It's good to know that your fine")

        elif "change my name to" in query:
            query = query.replace("change my name to", "")
            assname = query

        elif "change name" in query:
            speak("What would you like to call me, Dear ")
            asname = takeCommand()
            speak("Thanks for naming me")

        elif "your name" in query:
            speak("My friends call me")
            speak(asname)
            Text1 += "My friends call me " + asname + "\n"

        elif 'exit' in query:
            speak("Thanks for giving me your time")
            break

        elif "who made you" in query or "who created you" in query:
            speak("I have been created by tausif as a mini project")

        elif 'search' in query or 'play' in query:
            query = query.replace("search", "")
            query = query.replace("play", "")
            webbrowser.open(query)

        elif "who i am" in query:
            speak("If you talk then definitely your human.")

        elif "why you came to world" in query:
            speak("Thanks to tausif. I am just for the ")

        elif "who are you" in query:
            speak("I am your virtual assistant created by Tausif")

        elif 'reason for you' in query:
            speak("I was created as a Mini project by Tausif ")

        elif 'change background' in query:
            ctypes.windll.user32.SystemParametersInfoW(20,0,"Location of wallpaper",0)
            speak("Background changed successfully")
            Txet1 += "Background changed successfully\n"

        elif 'news' in query:
            NewsFromBBC()
            time.sleep(2)

        elif 'lock window' in query:
            speak("locking the device")
            ctypes.windll.user32.LockWorkStation()

        elif 'shutdown system' in query:
            speak("Hold On a Sec ! Your system is on its way to shut down")
            subprocess32.call('shutdown / p /f')

        elif "don't listen" in query or "stop listening" in query:
            speak("for how much time you want to stop zara from listening commands")
            a = int(takeCommand())
            time.sleep(a)
            Text1+=a
            
        elif "where is" in query:
            query = query.replace("where is", "")
            location = query
            speak("User asked to Locate")
            speak(location)
            webbrowser.open("https://www.google.nl / maps / place/" + location + "")

        elif "camera" in query or "take a photo" in query:
            ec.capture(0, "Jarvis Camera ", "img.jpg")

        elif "restart" in query:
            subprocess32.call(["shutdown", "/r"])

        elif "sleep" in query:
            speak("sleeping...")
            subprocess32.call("shutdown / h")

        elif "log off" in query or "sign out" in query:
            speak("Make sure all the application are closed before sign-out")
            time.sleep(5)
            subprocess32.call(["shutdown", "/l"])

        elif "write a note" in query:
            speak("What should i write, sir")
            note = takeCommand()
            file = open('jarvis.txt', 'w')
            file.write(note)

        elif "show note" in query:
            speak("Showing Notes")
            file = open("jarvis.txt", "r")
            Text1 += file.read() + "\n"
            speak(file.read(6))
            
        elif "zara" in query:
            wishMe()
            speak("zahra in your service Dear")
            speak(uname)

        elif "wikipedia" in query:
            webbrowser.open("wikipedia.com")

        elif "morning" in query:
            speak("A warm" + query)
            speak("How are you")
            
    return Text1
    

# Creating a GUI for user friendly ineraction with user 

In [ ]:
from tkinter import *
import os

Text1 = ''

def update_gui():
    user_name_input_area.delete(1.0, END)  # Clear existing content
    user_name_input_area.insert(INSERT, Text1)  # Update with new content
    root.after(10, update_gui)  # Schedule the update after 100 milliseconds

def assistant_logic():
    global Text1
    wishMe()
    username()
    root.after(10, update_gui)  # Schedule the initial update
    assistant()

# Create a tkinter window
root = Tk()

# Open window having dimension 500x500 (smaller size)
root.geometry('650x500')

# Load the background image
bg = PhotoImage(file="virtual-assistant.png", master=root)

# Calculate the center position for the background image

bg_x = (900 - bg.width()) // 2
bg_y = (900 - bg.height()) // 2

# Show image using label
label1 = Label(root, image=bg)
label1.place(x=bg_x, y=0)

# Create a Button
btn = Button(root, text='~Speak~', command=assistant_logic)

# Create a Text area
user_name_input_area = Text(root, width=30, height=31)  # Adjusted width and height
user_name_input_area.place(x=0, y=0)

btn.place(x=250, y=470)

# Set the position of the button on the top of the window.
root.title("Zahra - A Virtual Personal Assistant")
root.mainloop()
